### Домашнее задание к лекции «Понятие класса»
А.Петряшев (DS-28)

In [1]:
import requests

In [2]:
import json

In [74]:
valute = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')

Задание 1.

Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js



Не совсем понял, какое максимальное значение нужно получить, поэтому сделал 2 решения.

1.1.
Максимальное значение из предыдущего и текущего курсов конкретной валюты или всех валют сервиса.

In [94]:
user_input = input()

def valute_max(valuta=''):
    currency = valute.json()['Valute']
    for val, des in currency.items():
        if valuta != '':
            
            if val == valuta.upper():
                name = des['Name']
                max_v = max([des['Value'], des['Previous']])
                print(name, max_v)
            else: pass
        else:
            name = des['Name']
            max_v = max([des['Value'], des['Previous']])
            print(name, max_v)
valute_max(user_input)

usd
Доллар США 76.2491


1.2. 
Максимальное значение текущего курса валюты из всех валют сервиса.

In [91]:
def max_value():
    cur = valute.json()['Valute']
    all_rates ={}
    for val, des in cur.items():
        current_rate = des['Value']
        val_name = des['Name']
        all_rates[current_rate] = val_name
    #print(all_rates)
    max_rate = max(all_rates)
    name = all_rates[max_rate]
    print(name, max_rate)
max_value()        
        
    

СДР (специальные права заимствования) 109.0321


Задание 2.

Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах курсов валют (eur, usd итд) будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.


In [105]:
class Rate:
    def __init__(self, format_='value', diff=True):
        self.format = format_
        self.diff = diff
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return self.r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        
        если diff=True - находится разница значений 'Value'-'Previous' и выводится вместо курса валюты,
        в противном случае выводится текущий курс;
        
        diff не используется при выводе поной информации о валюте
        
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                if self.diff:
                    return response[currency]['Value'] - response[currency]['Previous']
                else:
                    return response[currency]['Value']
        
        return 'Error'
    
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR')
    
    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD')
    
    def brl(self):
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        return self.make_format('BRL')

In [103]:
r = Rate(format_='value', diff=True)

In [104]:
r.eur()

0.540600000000012

Задание 3.

Напишите класс Designer, который учитывает количество международных премий для дизайнеров (из презентации: “Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла”). Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять).

Класс Designer пишется по аналогии с классом Developer из материалов занятия. Комментарий про его условия Вика написала выше: “Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла”

In [305]:
class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority
        
        self.grade = 1
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)
    
    def check_if_it_is_time_for_upgrade(self):
        pass

Для решения предположил, что премиии могут быть в любой год. 
Так как балов за премию дают 2, а повышение грейда идет за каждые 7 баллов, то постарался реализовать такую логику: 

количество премий в год выбирается случайно, но не более одной в год - random.randint(0,1);

сначала повышается грейд за стаж, затем за балллы - учитывается накопление баллов и шкала для повышения (7, 14, 21,...).

In [109]:
import random

In [379]:
class Designer(Employee):
    def __init__(self, name, seniority, raiting = 4, prom_raiting=[0]):
        super().__init__(name, seniority)
        
        self.raiting = raiting
        self.prom_raiting = prom_raiting
    
    def check_if_it_is_time_raiting_for_upgrade(self):
        """
        prize_points = random.randint(0,1)*2 - подсчет квалификационных баллов за примемии, случайное определение кол-ва премий
        diff = self.raiting - self.prom_raiting[-1] - разница баллов последнего повышения и текущих
        points_for_grade = (self.raiting // 7) * 7 - баллы за которые должны повышаться грейды
           
        """
        prize_points = random.randint(0,1)*2
        self.raiting +=prize_points
        self.seniority += 1
        diff = self.raiting - self.prom_raiting[-1]
        points_for_grade = (self.raiting // 7) * 7
        
        if self.seniority % 5 == 0:
            
            self.grade_up()
            
        elif  diff >= 7 or self.raiting % 7 == 0:
            self.prom_raiting.append(points_for_grade)
            self.grade_up()
            
        #print('баллов за премию: ', prize_points, '//', 'всего баллов: ', self.raiting, '//', 'баллы на момент прома: ', self.prom_raiting, '//', 'разница: ', diff)
        
        return self.publish_grade()
        
        

In [377]:
alex= Designer('alex', 0)

In [378]:
for i in range(10):
    alex.check_if_it_is_time_raiting_for_upgrade()

баллов за премию:  0 // всего баллов:  4 // баллы на момент прома:  [0] // разница:  4
alex 1
баллов за премию:  2 // всего баллов:  6 // баллы на момент прома:  [0] // разница:  6
alex 1
баллов за премию:  0 // всего баллов:  6 // баллы на момент прома:  [0] // разница:  6
alex 1
баллов за премию:  0 // всего баллов:  6 // баллы на момент прома:  [0] // разница:  6
alex 1
баллов за премию:  2 // всего баллов:  8 // баллы на момент прома:  [0] // разница:  8
alex 2
баллов за премию:  0 // всего баллов:  8 // баллы на момент прома:  [0, 7] // разница:  8
alex 3
баллов за премию:  2 // всего баллов:  10 // баллы на момент прома:  [0, 7] // разница:  3
alex 3
баллов за премию:  0 // всего баллов:  10 // баллы на момент прома:  [0, 7] // разница:  3
alex 3
баллов за премию:  0 // всего баллов:  10 // баллы на момент прома:  [0, 7] // разница:  3
alex 3
баллов за премию:  2 // всего баллов:  12 // баллы на момент прома:  [0, 7] // разница:  5
alex 4


такую логику использовал для повышения грейда

In [381]:
t = 2
k =[0]
seniority = 0

for i in range(0,20):
   
    seniority +=1
  
    t+=2 
   
    d = t - k[-1] #разница баллов последнего повышения и текущих
 
    p_p = (t//7)*7 #баллы за которые должны повышаться грейды
   
    if seniority % 5 == 0:
        print('grade up +5year', 'in company', seniority, 'year', '//', p_p)
   
    elif d >= 7 or t % 7 == 0:
        k.append(p_p)
        print('grade up +7points', t, 'overall points', '//', p_p)
   
    else: print('still', t)
print(k)

still 4
still 6
grade up +7points 8 overall points // 7
still 10
grade up +5year in company 5 year // 7
grade up +7points 14 overall points // 14
still 16
still 18
still 20
grade up +5year in company 10 year // 21
grade up +7points 24 overall points // 21
still 26
grade up +7points 28 overall points // 28
still 30
grade up +5year in company 15 year // 28
still 34
grade up +7points 36 overall points // 35
still 38
still 40
grade up +5year in company 20 year // 42
[0, 7, 14, 21, 28, 35]
